# IMDB Random Forests Regression
![ImdbIcon](../images/imdbheader.jpg)

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\nolan_fur2pfn\.conda\envs\dsi\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
revenue_df = pd.read_csv('../data/revenue_df.csv')

In [5]:
revenue_df.columns

Index(['movie_title', 'year', 'tagline', 'plot', 'cast', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama',
       'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western',
       'total_votes', 'us_voters_votes', 'votes', 'votes_1', 'votes_2',
       'votes_3', 'votes_4', 'votes_5', 'votes_6', 'votes_7', 'votes_8',
       'votes_9', 'votes_10', 'popularity', 'director_score', 'actor_score',
       'actress_score', 'tagline_sentiment', 'plot_sentiment', 'imdb_score',
       'profitable', 'budget', 'revenue', 'budget_adj', 'revenue_adj'],
      dtype='object')

## Random Forests Regression

In [337]:
features = ['year', 'duration', 'Action', 'Adventure', 'Animation','Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'History', 
            'Horror',  'Musical', 'Mystery', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'us_voters_votes',  'votes_2',
            'votes_3', 'votes_4', 'votes_6', 'votes_7', 'votes_8', 'votes_9', 'votes_10', 'popularity', 'director_score', 
            'actor_score', 'tagline_sentiment', 'plot_sentiment', 'imdb_score', 'profitable']

In [338]:
X = revenue_df[features]
y = revenue_df['revenue']

In [339]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 42)

In [340]:
rf = RandomForestRegressor()

In [341]:
rf.fit(X_train, y_train)

RandomForestRegressor()

In [342]:
print("Training Score:", rf.score(X_train, y_train))
print(" ")
print("Testing Score:", rf.score(X_test, y_test))

Training Score: 0.952183935323503
 
Testing Score: 0.7566925318050755


In [343]:
print("Cross Val Score:", cross_val_score(rf, X_train, y_train, cv = 5).mean())

Cross Val Score: 0.6638934282928202


In [349]:
model_params = {}
count = 0

In [354]:
rf_params = {
    'max_depth': [14, 16, 18],
    'min_samples_leaf': [0, 1, 2],
    'min_samples_split': [1, 2, 3],
    'n_estimators': [100, 200, 300],
}

gs = GridSearchCV(rf, param_grid=rf_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient='index')

model_df

,max_depth,min_samples_leaf,min_samples_split,n_estimators,best_score,training_score,testing_score
model_1,2,1,2,200,0.510903,0.588115,0.578101
model_2,6,1,2,300,0.651673,0.859662,0.747793
model_3,10,1,3,300,0.665289,0.926700,0.747227
model_4,14,1,2,200,0.668167,0.946944,0.757747
model_5,18,1,2,100,0.671781,0.949832,0.750838


## AdaBoostRegressor

In [356]:
model_params_two = {}
count_two = 0

In [359]:
ada = AdaBoostRegressor(base_estimator=RandomForestRegressor())

ada_params = {
    'base_estimator__max_depth': [3, 4, 5],
    'learning_rate': [.85, .90, .95],
    'n_estimators': [150, 200, 250],
}

gs_two = GridSearchCV(ada, param_grid=ada_params, cv=3, n_jobs = 12)

gs_two.fit(X_train, y_train)

count_two += 1

gs_two.best_params_['best_score'] = gs_two.best_score_

gs_two.best_params_['training_score'] = gs_two.score(X_train, y_train)

gs_two.best_params_['testing_score'] = gs_two.score(X_test, y_test)

model_params_two[f'model_{count_two}'] = gs_two.best_params_

model_df_two = pd.DataFrame.from_dict(model_params_two, orient='index')

model_df_two

,base_estimator__max_depth,learning_rate,n_estimators,best_score,training_score,testing_score
model_1,3,0.9,100,0.546156,0.701877,0.630812
model_2,4,0.9,200,0.594956,0.794102,0.661616
model_3,4,0.9,200,0.584980,0.622155,0.361693
